# Week 9 : Lab 
 ## Functional: Recursion, comprehensions and first-class functions
##### CS1P - University of Glasgow - John Williamson - 2023/2024

In [ ]:
from utils.tick import tick
from utils.recursion_check import test_recursion
import ast
from utils.vector_checks import check_ast, lines_ast
from binascii import crc32

def check_string(x, test_crc):    
    crc = crc32(str(x).encode())
    print(hex(crc))
    return crc==test_crc



## No B part

There is only an A part to this lab, but the questions are harder than previous A parts.



## Purpose of this lab
This lab will get you up to speed on:
* Using comprehensions
* Using closures
* Recursive solutions to problems
* Functional approaches to solving programming problems.
* Higher-order functions



## A. Quick problems

### A.1 Checkerboard
A square checkerboard pattern of size 4 looks like this:

        ◯ ⚫ ◯ ⚫
        ⚫ ◯ ⚫ ◯
        ◯ ⚫ ◯ ⚫
        ⚫ ◯ ⚫ ◯
    
Using comprehensions write a function `checker8()` that has *one line of code -- a single statement* that returns such a checkerboard of size 8. Hint: you might want to use `join`.

Note that the circles are just ordinary characters. You can copy and paste them into your code.

In [ ]:
## Solution
def checker8():    
    return("\n".join([" ".join(["◯" if (i+j)%2==0 else "⚫" for i in range (8)]) for j in range(8)]))


In [ ]:
## Tests
print(checker8())
with tick():
    assert lines_ast(checker8)==1, "Not one line!"
    assert check_ast(checker8, forbidden_nodes=[ast.For, ast.While], required_nodes=[ast.ListComp]), "Is that a comprehension?"
    assert check_string(checker8(), 0xe5c7b16a)

Extend this to write a function `checkerboard(n, on, off)` which takes the size of the (square) checkerboard `n` and the symbols to use as `on` and `off`, defaulting to the `◯` and `⚫` used above. This should **return** the string, and not print it out.

In [ ]:
## Solution
def checkerboard(n, on='◯', off='⚫'):
    return "\n".join([" ".join([on if (i+j)%2==0 else off for i in range (n)])
          for j in range(n)])


In [ ]:
## Tests
from textwrap import dedent

with tick():
    assert lines_ast(checkerboard)==1, "Still got to do it on the one line!"
    assert check_ast(checkerboard, forbidden_nodes=[ast.For, ast.While], required_nodes=[ast.ListComp]), "Uhh, where's the comprehension?"
    assert checkerboard(3) == dedent("""
    ◯ ⚫ ◯
    ⚫ ◯ ⚫
    ◯ ⚫ ◯
    """).strip()

    assert checkerboard(3, 'o', 'x') ==dedent("""
    o x o
    x o x
    o x o
    """).strip()
    
    assert checkerboard(3, 'N', 'N') ==dedent("""
    N N N
    N N N
    N N N    
    """).strip()

    # is it cursed?
    assert checkerboard(7, "🦉", "💍")==dedent("""
    🦉 💍 🦉 💍 🦉 💍 🦉
    💍 🦉 💍 🦉 💍 🦉 💍
    🦉 💍 🦉 💍 🦉 💍 🦉
    💍 🦉 💍 🦉 💍 🦉 💍
    🦉 💍 🦉 💍 🦉 💍 🦉
    💍 🦉 💍 🦉 💍 🦉 💍
    🦉 💍 🦉 💍 🦉 💍 🦉
    """).strip()

    assert checkerboard(4, "⍝", "⌾")  == dedent("""
    ⍝ ⌾ ⍝ ⌾
    ⌾ ⍝ ⌾ ⍝
    ⍝ ⌾ ⍝ ⌾
    ⌾ ⍝ ⌾ ⍝
    """).strip()

## A.2 Compose

Write a function `compose(fn_a, fn_b)` which **returns a function** that applies `fn_a` then `fn_b` to a value, using `lambda`. Assume `fn_a` and `fn_b` take exactly one argument.


In [ ]:
## Solution
def compose(fn_a, fn_b):
    return lambda x: fn_b(fn_a(x))


In [ ]:
## Tests

add_1 = lambda x: x + 1
mul_3 = lambda x: x * 3
identity = lambda x: x
first = lambda x: x[0]

with tick():    
    assert check_ast(compose, forbidden_nodes=[ast.Assign], required_nodes=[ast.Lambda])
    assert compose(add_1, mul_3)(5) == 18
    assert compose(mul_3, add_1)(5) == 16
    assert compose(mul_3, identity)(5) == 15
    assert compose(identity, identity)("hey") == "hey"
    assert compose(identity, identity)(["hey"]) == ["hey"]
    assert compose(first, first)([["beta", "prime"]]) == "beta"
    

### A.3 Recursive dot product
Write a function `dot(a,b)` which takes two lists `a` and `b` of equal length, and computes the dot product; that is the sum of the elementwise product of each element from `a` and `b`. For example `dot([0,1,2], [3,4,5]) = 0*3 + 1*4 + 2*5 = 14`.

Write this **recursively** without using `for`, `while` or any `comprehensions`, or any numerical arrays.

In [ ]:
## Solution
def dot(a,b):
    if len(a)==0:
        return 0
    else:
        return a[0]*b[0] + dot(a[1:], b[1:])

In [ ]:
with tick():
    assert check_ast(dot, forbidden_nodes=[ast.For, ast.While, ast.ListComp, ast.SetComp, ast.DictComp], required_nodes=[])
    assert test_recursion(lambda: dot([0,1,2], [3,4,5])), "Oi, I said recursive!"
    assert(dot([0,1,2], [3,4,5])==14)
    assert(dot([0,-1,2], [3,4,5])==6)
    assert(dot([], [])==0)
    assert(dot([1], [8])==8)
    assert(dot([0,1,2,5], [3,4,5,9])==59)

### A.4 Accumulate

Write a function recursive `accumulate(fn, seq)` that does an operation similar to `reduce`, in that it puts a function "in between" consecutive elements of a list, but unlike `reduce` keeps all of the intermediate results. (This operation is sometimes called "scan" in other languages)

For example

    accumulate + 1 2 3 4
    = 1  
      ⇓  
      1 + 2  3  4        
        ⇓  
        3 + 3  4           
          ⇓ 
          6 + 4
            ⇓ 
            10
             
     = [1,3,6,10]

This means that the accumulation of a sequence with addition produces the cumulative sum (or running total) instead of the sum, as `reduce` would. The accumulation of a sequence with the max function produces the running maximum.

* `accumulate(lambda x,y:x+y, [1,2,3,4])` would return `[1, 1+2, 1+2+3, 1+2+3+4]` = `[1,3,6,10]`
* `accumulate(max, [1,4,3,9, 2])` would return `[1, 4, 4, 9, 9]`

The accumulation of the empty list is the empty list.




In [ ]:
## Solution

def accumulate(fn, seq):
    if len(seq)<2:
        return seq    
    else:
        return [seq[0]] + accumulate(fn, [fn(seq[0], seq[1])] + seq[2:])


In [ ]:
## Tests
with tick():
    assert(accumulate(lambda x,y:x+y, [1,2,3])==[1,3,6])
    assert(accumulate(lambda x,y:x-y, [1,2,3])==[1,-1,-4])
    assert(accumulate(max, [1,4,3,9,2])==[1,4,4,9,9])
    assert(accumulate(max, [])==[])
    assert(accumulate(max, [1])==[1])  
    assert check_ast(accumulate, forbidden_nodes=[ast.For, ast.While, ast.ListComp, ast.SetComp, ast.DictComp], required_nodes=[])

### A.5 SKIing

You can get the equivalent of nested functions using `lambda`, but it is shorter:

```python
def f(x):
    def g(y):
      return x + y
    return g
```

is the same as:

```python
f = lambda x: lambda y: x + y
```

you can read it as if it were grouped like this:

```python
f = lambda x: (lambda y: (x + y))
```

You would read this in English as "a function that takes x and returns a function that takes y, and returns x+y".


**Using lambda**, write three functions called I, K and S which are defined so that:

* I(x) returns x: `I(x) = x`
* K(x) returns a function which takes y and returns x: `K(x)(y) = x`
* S(x): which takes a function `x`, and returns a function which takes a function `y` which returns a function which takes a `z` and returns `(x(z))(y(z))`: `S(x)(y)(z) = (x(z))(y(z))`

In [ ]:
## Solution  

I = lambda x: x
K = lambda x: lambda y: x
S = lambda x: lambda y: lambda z: (x(z))(y(z))


In [ ]:
## Tests
with tick():
    
    assert "<lambda>" in str(S)
    assert "<lambda>" in str(K)
    assert "<lambda>" in str(I)
    assert I(1) == 1
    assert I(0) == 0
    assert I(I) == I
    assert K(0)(1) == 0
    assert K(1)(0) == 1
    assert K(K)(I) == K
    assert S(I)(I)(I) == I
    assert S(K)(I)(I) == I
    assert S(K)(I)(K) == K
    assert S(I)(K)(I)(1)(2) == 2
    
    assert "lambda" in str(I)
    assert "lambda" in str(K(1))
    assert "lambda" in str(K(I))
    assert "lambda" in str(S(1))

    bluebird = S(K(S))(K)
    double_list = bluebird(lambda x:[x])(lambda x:x*2)
    list_double = bluebird(lambda x:x*2)(lambda x:[x])
    assert double_list(2) == [4]
    assert list_double(2) == [2, 2]



#### Advanced stuff (you don't need to read this part of A.5 => go to A.6 if you're not interested)

You've just implemented the entire [SKI combinator calculus](https://en.wikipedia.org/wiki/SKI_combinator_calculus). You don't need to understand this, but these three functions S, K, I are a powerful way of representing *any program*. 


For example, we can implement logical AND, NOT and OR:

In [ ]:
TRUE = K # True
FALSE = K(I) # False

AND = S(S)(K(K(FALSE))) # And
OR = S(I)(K(TRUE)) # Or 
NOT= S(S(I)(K(FALSE)))(K(TRUE)) # Not

def print_bool(f):
    # print an "SKI" Boolean out
    print(f(True)(False))

# compute AND of two functions    
print_bool(AND(FALSE)(FALSE))    
print_bool(AND(TRUE)(FALSE))    
print_bool(AND(FALSE)(TRUE))
print_bool(AND(TRUE)(TRUE))

# = not(t and (t or f))
print_bool(NOT(OR(TRUE)(AND(TRUE)(FALSE))))



We can make this "cleaner" (sort of), with some Python trickery:

In [ ]:
class λ:
    def __init__(self, fn, name=None):
        self.name = name
        self.fn = fn
        
    def __repr__(self):
        return self.name
    
    def __call__(self, args):
        return self.fn(args)
                       
𝕋 = λ(K, "𝕋")
𝔽 = λ(K(I), "𝔽")

ᐱ = S(S)(K(K(𝔽))) # And
ᐯ = S(I)(K(𝕋)) # Or 
ᓓ = S(S(I)(K(𝔽)))(K(𝕋)) # Not

# compute AND of two functions    
print(ᐱ(𝔽)(𝔽))    
print(ᐱ(𝕋)(𝔽))    
print(ᐱ(𝔽)(𝕋))
print(ᐱ(𝕋)(𝕋))

# = not(t and (t or f))
print(ᓓ(ᐱ(𝕋)(ᐯ(𝕋)(𝔽))))


### A.6 Collatz closure
Write a function `collatz(n)` which when called *returns a new function*. Each time the returned function is called it computes the update

* `n = 3 * n + 1` if `n` is odd 
* `n = n // 2` if n is even 

and returns the *previous* value `n`. Calling the function multiple times will return different values, and should iterate through this sequence using these rules -- see the test below for how this might work. Do not use any form of global variable.

In [ ]:
## Solution
def collatz(n):
    i_n = n
    def update_collatz():        
        nonlocal i_n
        old_n = i_n
        i_n = 3 * i_n + 1 if i_n%2==1 else i_n // 2
        return old_n
    return update_collatz


In [ ]:
## Test
with tick():
    assert check_ast(collatz, forbidden_nodes=[ast.Global], required_nodes=[])
    fn = collatz(241)
    seq = [fn() for i in range(22)]
    assert(seq==[241, 724, 362, 181, 544, 272, 136, 68, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1])

### A.7 Length of strings
Write a function `len_not_a()`, which takes a list of strings, and returns a list of the length of each of the strings, *except* if the string contains a character "a", in which case the length should be omitted entirely from the returned list.

For example, `["cat", "dog", "fifteen"]` should become [3, 7] (`"cat"` is omitted from the returned list)
    
Use a comprehension. **Do not use a for or while loop.**

In [ ]:
## Solution
def len_not_a(string_list):
    return [len(a) for a in string_list if "a" not in a]

In [ ]:
## Tests
with  tick():
    assert check_ast(len_not_a, forbidden_nodes=[ast.For, ast.While], required_nodes=[ast.ListComp])
    assert(len_not_a([])==[])    
    assert(len_not_a(["cat", "dog", "fifteen"])==[3,7])
    assert(len_not_a(["cat", "dog", "cat", "fiftaan"])==[3])
    assert(len_not_a(["b", "a", "a", "a"])==[1])
    assert(len_not_a(["b", "ar", "ra", "ar", 'dly'])==[1, 3])

## A.8 Comprehensify

There are three loops below written using a standard `for` loop. Rewrite them as a **single comprehension** inside a `print()` call. Your comprehension version should behave exactly the same as the original code.


In [ ]:
# This code sets up the data we will use
# run this cell first
metals = ["silver", "bronze", 
          "steel", "gold", 
          "chrome", "lead",
          "copper", "aluminium"]

In [ ]:
# A.2.1
squares = []
for i in range(10):
    squares.append(i*i)
print(squares)

In [ ]:
## Solution
print([i*i for i in range(10)])

In [ ]:
# A.2.2
metals_sorted = sorted(metals)
metals_list = []
for metal in metals_sorted:
    if "ium" not in metal:        
        new_title = metal.title()+" spoon"
        metals_list.append(new_title)
for metal in metals_list:
    print(metal)

In [ ]:
## Solution
print("\n".join([metal.title()+" spoon" 
                 for metal in sorted(metals) 
                 if "ium" not in metal]))

In [ ]:
# A.2.3
# This one is trickier, but still defintely doable.
# If you struggle, move on and do the next problem.

# Dictionary mapping metal names to number of vowels in
# that name
metal_vowels = {}
for metal in metals:
    vowels = 0
    for char in metal:
        if char in 'aeiou':
            vowels += 1
    metal_vowels[metal] = vowels
        
print(metal_vowels)

In [ ]:
## Solution
print({metal:sum([1 for char in metal if char in 'aeiou'])
       for metal in metals})

### A.9 Using *
Python functions can take a variable number of arguments. If we write a function like below, with a *single* parameter beginning with `*` the parameter will be a *list* of the arguments passed.  [See here for more details](https://www.saltycrane.com/blog/2008/01/how-to-use-args-and-kwargs-in-python/)

In [ ]:
def multi(*args): # args will be a list
    print("I was called with {n} arguments".format(n=len(args)))
    for arg in args:
        print("\t", arg)
    print()
    
multi(1)
multi("hey", "there")
multi([1,2,3], [4,5,6], 'oomf')

Likewise, we can pass a *list* of arguments to any function if we use the `*` symbol (the "splat" operator) in the argument list. This operation is called **apply** in other languages. It applies the function to a sequence of arguments:

In [ ]:
def print_sum(a,b):
    print(a+b)
    

In [ ]:
print_sum(4,5)

In [ ]:
# exactly the same as before
print_sum(*[4,5]) # call with a list

In [ ]:
a  = [2,9]
print_sum(*a) # call with a list in a variable

In [ ]:
bad = [1,2,3]
print_sum(*bad) # this will cause an error; too many arguments


Write a function `backwards(fn)` that "wraps" a function to reverse the order of the arguments `fn` will receive. `backwards` takes a function returns *a new function* which will work the same as `fn` but will receive the arguments in reverse order. 

In [ ]:
def space_join(*args):
    return " ".join(args)

def sub(a,b):
    return a-b

print(space_join("alpha", "bravo", "charlie"))
print(sub(5,10))

In [ ]:
## Solution
def backwards(fn):
    def wrapped_fn(*args):
        return fn(*reversed(args))
    return wrapped_fn

In [ ]:
## Tests
with tick():
    backwards_join = backwards(space_join)
    assert(backwards_join("alpha", "bravo", "charlie")=="charlie bravo alpha")
    backwards_sub = backwards(sub)
    assert(backwards_sub(5, 10)==5)

## A.10 Getting closure
Write a function called `make_storybot(name)`. It should *return* a function which takes no arguments. Each time the *returned function* is called, it should print out a message, *and return a function* that will print out the *next* line, and so on.

The messages printed out on each call, should be as follows, assuming `name` was `John` when `make_storybot()` was called:

    (first call)   "Once upon a time."
    (second call)  "There was someone known as John."
    (third call)   "John lived in the distant mountains."
    (fourth call)  "And ate seagulls."
    (fifth call)   "John did not enjoy eating seagulls."
    (sixth call)   "What are seagulls?"
    (seventh and all subsequent calls) "We just don't know."
    
Do **not** use any global variables. `lambda` is useful in this solution.
*Feel free to adapt the story to a more uplifting one if you so desire.*


In [ ]:
## Solution
def make_storybot(name):
    responses = ["Once upon a time.", 
                 "There was someone known as {name}.", 
                 "{name} lived in the distant mountains.", 
                 "And ate seagulls.", 
                 "{name} did not enjoy eating seagulls.", 
                 "What are seagulls?",
                 "We just don't know."]
    
    # have to define an inner function which will be the closure we return
    def chat(responses):
        # print the response and move to the next response
        print(responses[0].format(name=name))     
        if len(responses)==1:
            return lambda: chat([responses[0]])
        else:
            return lambda: chat(responses[1:])                            
    return lambda: chat(responses)  # this is critical!

In [ ]:
## Test
assert check_ast(make_storybot, forbidden_nodes=[ast.Global, ast.Nonlocal], required_nodes=[])

name = 'John' # put your name here
chatter = make_storybot(name)

# these should print out different messages, as above
chatter = chatter()
chatter = chatter()
chatter = chatter()
chatter = chatter()
chatter = chatter()
chatter = chatter()
chatter = chatter()
chatter = chatter()
chatter = chatter()

The output for the test above should look like:
    
        Once upon a time.
        There was someone known as John.
        John lived in the distant mountains.
        And ate seagulls.
        John did not enjoy eating seagulls.
        What are seagulls?
        We just don't know.
        We just don't know.
        We just don't know.
        
But with the correct name substituted in.        

## A.10 List transpose
Write a function `transpose(l)` that takes a matrix *represented as a list of lists* (**NOT as NumPy array**) and transposes it; that is exchanges rows and columns. Hint: `zip` and `*args` are useful here.

`[[1,2,3], [4,5,6], [7,8,9]]` should become `[[1,4,7], [2,5,8], [3,6,9]]` when transposed. Your function should also work for non-square arrays, where rows and columns are *not* equal. `[[1,2], [3,4], [5,6], [7,8]]` should become 
`[[1,3,5,7], [2,4,6,8]]` when transposed.

In [ ]:
## Solution
def transpose(l):
    # easy when you know how...
    return [list(row) for row in zip(*l)]

In [ ]:
## Tests
with tick():
    assert(transpose([[1,2,3], [4,5,6], [7,8,9]])==[[1,4,7], [2,5,8], [3,6,9]])
    assert(transpose([[1,2], [3,4], [5,6], [7,8]])==[[1,3,5,7], [2,4,6,8]])
    assert(transpose([])==[])

## A.11 Currying
A function can be **curried** to convert it from a multi-parameter function `f(a,b,c,...)` into a single parameter function `g(x)`. Curry does not refer to the Indian style of cooking, but to the influential computer scientist **Haskell Curry**.
<img src="imgs/HaskellBCurry.jpg">

*Image: Haskell B. Curry*

The single parameter function has to be called multiple times to pass each of the arguments in turn. Only after *all* of its parameters are ready is the function executed and its value returned. This can be useful in reasoning about the behaviour of programs, because *every* function call can then be reduced to a sequence of single argument calls.

Imagine this function:

In [ ]:
def mul_and_add(a,b,c):
    return a * b + c

This can be called in the normal way like this:

In [ ]:
mul_and_add(2,3,4)

The **curried** version would be called like this:

    curried_mul_and_add(2)(3)(4)
    
Only the **last** call would do the computation and return the result.
    

### Task
Write a function `curry(fn, n)` that will curry *any* function in this way. `n` should specify the number of expected arguments to `fn` 

    curried_mul_and_add = curry(mul_and_add, 3)
    
should produce a function like the one described above.
    


In [ ]:
## Solution
# many ways of doing this; this one is a little ugly but does the trick
def curry(fn, n):         
    def curried_version(args):        
        return fn(*args) if len(args)==n else lambda x:curried_version(args+[x])        
    return curried_version([])

In [ ]:
curried_mul_and_add = curry(mul_and_add, 3)

with tick():
    assert curried_mul_and_add(2)(3)(4)==10
    assert curried_mul_and_add(4)(5)(6)==26
    
curried_add = curry(lambda x,y: x+y, 2)

with tick():
    assert curried_add(2)(5) == 7
    assert curried_add(-10)(10) == 0
    

## A.12 Higher-order functions
Write a function `per_element(fns)`. This takes a list of functions `fns`, and returns **a new function** `f(list_of_lists)` that will apply each function in `fns` to the corresponding sublist element in the list-of-lists **list_of_lists**. 

So if `fns` is a pair of functions, the returned function will transform any list of pairs, applying `fn[0]` to the first element of each pair, and `fn[1]` to the second element of each pair.

Examples:
* `per_element([len])` should return a function, which when called with `[["one"], ["theory"], ["is"]]`, should return [[3],[6],[2]]

        f = per_element([len]) 
        print f([["one"], ["theory"], ["is"]])

        [[3], [6], [2]]

* `per_element([lambda x:x*2, lambda x:"constant"])` should return a function, which when called with `[[1,1], [2,1], [4,2]]` should return `[[2,"constant"], [4,"constant"], [8, "constant"]]`

        g = per_element([lambda x:x*2, lambda x:"constant"])
        print g([[1,1], [2,1], [4,2]])

        [[2,"constant"], [4,"constant"], [8, "constant"]]

Make sure you return a **function** that will do the computation in the future. Don't do the computation itself inside `per_element`!


In [ ]:
## Solution
def per_element(fns):
    def f(list_of_lists):
        return [[fns[i](g[i]) for i in range(len(fns))] 
                for g in list_of_lists]
    return f

In [ ]:
## Tests
with tick():
    f = per_element([len]) 
    assert(f([["one"], ["theory"], ["is"]])==[[3],[6],[2]])
    g = per_element([lambda x:x*2, lambda x:x])
    assert(g([[1,1], [2,1], [4,2]])==[[2, 1], [4, 1], [8, 2]])
    print("All OK!")

----------


# C: Optional problems

These are optional problems. You do not need to attempt any of this section. **THERE IS NO B PROBLEM IN THIS LAB!**

## C.1 Accessors
Accessing fields deeply nested inside dictionaries or lists -- as in the adventure game of Unit 5 -- can be irritating and unergonomic. Helper functions can make this much easier. We can write a *general* helper function to help pull out data from these structures.

Write a function `getter(*attrs)` that will take any number of arguments. These arguments are a sequence of keys (or integer indices). `getter` **returns a function** that accesses those keys/indices in order, by chaining each indexing operation in sequence (as in `d["key1"]["key2"][...` etc.)

> For example, `fn = getter("state", "current_room")` would return a function which when called like `fn(d)` would be equivalent to `d["state"]["current_room"]`.

If `None` appears in the arguments, the function *returned* should require as many additional arguments as there are `None`s when it is called, which will "fill in the blanks" to access a nested data strucure.

> For example, `fn = getter("state", None)` would return a function which when called like `fn(d, "current_room")` would be equivalent to `d["state"]["current_room"]`.
it is called, which will "fill in the blanks" to access a nested data strucure.

> Likewise, `fn = getter(None, "current_room")` would return a function which when called like `fn(d, "state")` would be equivalent to `d["state"]["current_room"]`.

Look at this data structure, describing courses at Glasgow:

In [ ]:
courses = [
    {
        "college": "CoSE",
        "course": {
            "code": "COMPSCI1001",
            "title": "CS1P",
            "credits": 20,
            "staff": {
                "lecturer": [
                    {"name": {"first": "John", "last": "Williamson"}, "semester": 1},
                    {"name": {"first": "Sofiat", "last": "Olaosebikan"}, "semester": 2},
                ],
                "tutors": [
                    {"group": "LB01", "name": {"first": "John", "last": "Williamson"}},
                    {"group": "LB02", "name": {"first": "Jack", "last": "Parkinson"}},
                    {"group": "LB03", "name": {"first": "Alex", "last": "Pancheva"}},
                ],
            },
        }
    }
]

`getter()` should work as follows:

* `fn = getter(0, "course", "title")` would return a function `fn` that would extract element "CS1P" when called on `courses`
* `fn = getter(0, "course", "staff", "lecturer", 1, "name", "first])` would return a function that would look up the value `"Sofiat"` when called on `courses`
* `fn = getter(None, "course", "code"])` would return a function that would take an extra argument, giving the index to look up. So, calling  `fn(courses, 0)` would return `COMPSCI1001`.

In [ ]:
## Solution 

# simple version
def getter(*attrs):    
    def accessor(data, *args):   
        args = list(args)
        for i, attr in enumerate(attrs):
            if attr is None:
                attr = args.pop(0)
            data = data[attr]
        return data
    return accessor



In [ ]:
with tick():
    
    assert getter(0)(courses)==courses[0]
    assert getter(0, "course", "title")(courses)=='CS1P'
    assert getter(0, "college")(courses)=='CoSE'
    assert getter(0, "course", "staff", "tutors", None, "name", "first")(courses, 0)=="John"
    assert getter(0, "course", "staff", "tutors", None, "name", None)(courses, 1, "last")=="Parkinson"
    assert getter(0, "course", "staff", "tutors", None, None)(courses, 2, "group")=="LB03"
    

## C.1.2 Extended accessors

* If `[]` appears in the argument list to `getter`, and the item at that level is a list, a list of results should be collected, such that the tests below pass.

    * For example `fn = getter(0, "course", "staff", "lecturer", [], "name", "first])` would return `["John", "Sofiat"]` when `fn(courses)` is called.
    * For example `fn = getter([], "course", None, "lecturer", [], "name", "first])` would return `[["John", "Sofiat"]]` when `fn(courses, "staff")` is called.
    
* (slightly harder) Likewise for `{}`, collecting all of the elements into a dictionary.



In [ ]:
## Solution

# full version with all optional extensions
def getter(*attrs):    
    def accessor(data, *args):   
        args = list(args)
        
        for i, attr in enumerate(attrs):
            # optional extension
            acc = getter(*attrs[i+1:])          
            if attr==[]:                
                # collect results
                if type(data)==type([]):                                                                      
                    return [acc(d, *args) for d in data]
            elif attr=={}:                
                if type(data)==type({}):                           
                    return {d: acc(data[d], *args) for d in data}
            else:
                # iterate over results
                if type(attr)==list:                    
                    return [acc(data[a], *args) for a in attr]
                elif type(attr)==set:
                    return {a:acc(data[a], *args) for a in attr}
                else:
                    # placeholder
                    if attr is None:
                        attr = args.pop(0)
        
                    data = data[attr]
        return data
    return accessor

In [ ]:
with tick():
    print("For list collection")
    assert ((getter([], "college")(courses))==['CoSE'])
    assert (getter(0, "course", "staff", "tutors", [], "name", "first")(courses))==['John', 'Jack', 'Alex']
    assert (getter(0, "course", "staff", "tutors", [], "name", None)(courses, "first"))==['John', 'Jack', 'Alex']
    assert (getter([], "course", "staff", "tutors", [], "name", None)(courses, "first"))==[['John', 'Jack', 'Alex']]

with tick():        
    print("For dictionary collection")
    assert getter(0, "course", "staff", "tutors", [], "name", {})(courses) == [{'first': 'John', 'last': 'Williamson'}, {'first': 'Jack', 'last': 'Parkinson'}, {'first': 'Alex', 'last': 'Pancheva'}]
    assert getter(0, {})(courses)["college"] == "CoSE"

## C.1.3 Further optional extension :)

* If a sequence of values is passed for an argument (e.g. `["first", "last"]`) instead of a string or integer, each of those values should be collected into a list and returned.

For example,

    d = {"pets":
            [
                {"type":"cat",
                 "name":"cooper"
                 "temperament":"bad"
                 }
             ]
        }
    
    fn = getter("pets", 0, ["type", "name"])
    fn(d)
    
should return ["cat", "cooper"]
    
* Likewise, if a set of values is passed for an argument (e.g. `{"first", "last"}`) instead of a string or integer, each of those values should be collected into a dictionary.

    fn = getter("pets", 0, {"temperament", "name"})
    fn(d)
    
should return

        {"temperament":"bad", "name":"cooper"}

You don't have to worry about `None` appearing in these sets or lists.

In [ ]:
with tick():
    assert getter(0, "course", "staff", ["tutors", "lecturer"], [], "name", "first")(
        courses
    ) == [["John", "Jack", "Alex"], ["John", "Sofiat"]]
    assert getter(0, "course", "staff", {"tutors", "lecturer"}, [], "name", "first")(
        courses
    ) == {"tutors": ["John", "Jack", "Alex"], "lecturer": ["John", "Sofiat"]}
    assert getter([], None, "staff", {"tutors", "lecturer"}, [], "name", "first")(
        courses, "course"
    ) == [{"tutors": ["John", "Jack", "Alex"], "lecturer": ["John", "Sofiat"]}]
    assert(getter([], None, "staff", {"tutors", "lecturer"}, 0, "name", "first", None)(courses, "course", 0))== [{"lecturer":"J", "tutors":"J"}]
    assert(getter([], None, "staff", {"tutors", "lecturer"}, 0, "name", "first", None)(courses, "course", 0))== [{"lecturer":"J", "tutors":"J"}]
    

## C.2 Merge sort

How might we use recursion to **sort** things? There is a (relatively efficient) algorithm for sorting that is very easy to implement with recursion. This is **merge sort**. The basic principle is this:

### Merge rule
* If I have two sequences A=[a1, a2, a3, ...] and B=[b1, b2, b3, ...] and **both A and B are already in order**
* Then merging them to make a new sequence C only involves:
  * looking at *only* the first element of A and B and choosing the smaller one, 
  * removing it and adding it to C, 
  * then looking back at the new A and B, removing those smallest of the two and so on, 
  * ...until all elements of A and B are exhausted and C is full.

For example:

    A = 6 10 24 31 
    B = 1 9 32
    
    A = 6 10 24 31
    B = [1] 9 32
    C = 1
    
    A = [6] 10 24 31
    B = 9 32
    C = 1 6
    
    A = 10 24 31
    B = [9] 32
    C = 1 6 9
    
    A = [10] 24 31
    B = 32
    C = 1 6 9 10
    
    A = [24] 31
    B = 32
    C = 1 6 9 10 24
    
    
    A = [31]
    B = 32
    C = 1 6 9 10 24 31
    
    A = 
    B = [32]
    C = 1 6 9 10 24 31 32
    
    ---
    Finally:
    C = 1 6 9 10 24 31 32

So if we have two in order sequences, we can merge them into one in-order sequence in one pass. 

### Split rule
The next realisation is that we can "divide and conquer". In other words, we can take a sequence, and recursively split in half to get smaller and smaller chunks. 

    [ 10 6 24 31 9 1 32 ]
    
    [ 10 6 24 31 ] [ 9 1 32 ]
    
    [ 10 6 ] [ 24 31 ] [ 9 1 ] [ 32 ]

* If we end with a chunk of size 1, then it is in order by definition
* If we end up with a chunk of size 2, then we can just swap it if it is out of order.


        [ 10 6 ] [ 24 31 ] [ 9 1 ] [ 32 ]
            x        |        x       |
        [ 6 10 ] [ 24 31 ] [ 1 9 ] [ 32 ]

We can then merge all of these chunks together to get larger chunks, using the merge rule above. This will give us a sequence of larger chunks... which we can merge with the merge rule, until we have one sequence at the end. This will now be the entire original sequence in order; and this only takes `O(n log n)` steps to complete.

    [ 6 10 ] [ 24 31 ] [ 1 9 ] [ 32 ]
             
    [ 6 10 24 31 ] [ 1 9 32 ]
    
    [ 1 6 9 10 24 31 32 ]

### Task

* (a) Write `merge_sort(l, [key])` that will merge sort a list *taking an **optional** `key` parameter, just like sorted()*. 
* (b) Write tests that will verify that your function works the same as Python's built in `sorted()`. 


**Do not mutate *any* data or use any form of iteration!**


In [ ]:
## Solution

def merge(a, b, key):
    if len(a)==0:
        return b
    if len(b)==0:
        return  a
    if key(a[0])<key(b[0]):
        return [a[0]] + merge(a[1:], b, key)
    else:
        return [b[0]] + merge(a, b[1:], key)
    
def merge_sort(l, key=lambda x:x):        
    if len(l)<2: 
        return l
    else:
        n = len(l)//2
        return merge(merge_sort(l[:n], key), merge_sort(l[n:], key), key)
        


In [ ]:
        
## Tests

assert merge_sort([]) == sorted([])
assert merge_sort([1]) == sorted([1])
assert merge_sort(["a"]) == sorted(["a"])
assert merge_sort([1,2]) == sorted([1,2])
assert merge_sort([2,1]) == sorted([2,1])
assert merge_sort([3,2,1]) == sorted([3, 2,1])
assert merge_sort([1,2,3]) == sorted([1, 2,3])
assert merge_sort([1,2,3], key=lambda x:-x) == sorted([1, 2,3], key=lambda x:-x)
assert merge_sort([1,2,3,4], key=lambda x:-x) == sorted([1, 2,3,4], key=lambda x:-x)
assert merge_sort([4,1,2,3]) == sorted([4, 1,2,3])
assert merge_sort([[4,1],[2,3]]) == sorted([[4, 1],[2,3]])

## fuzz
import random
for i in range(100):
    n = random.randint(0, 20)
    l = [random.randint(-1000, 1000) for i in range(n)]
    assert merge_sort(l) == sorted(l)
    assert merge_sort(l, key=lambda x:x/2) == sorted(l, key=lambda x:x/2)
    

In [ ]:
# should print [3,2,1]
merge_sort([2,1,3], key=lambda x:-x)

## C.3 The lambda calculus - λ the ultimate

The *lambda calculus* is a representation of computation originally derived by Alonzo Church. It reduces *all* computation to simple functions of one argument. It is the reason that anonymous functions are called *lambda* in Python.

<img src="imgs/Alonzo_Church.jpg">

*Image: Alonzo Church*

We will show that we can do *anything* with just functions that take one argument and return one argument. Alonzo Church proved that [every computation can be written this way in 1936](https://www.ics.uci.edu/~lopes/teaching/inf212W12/readings/church.pdf) -- before there were any computers. He went on to supervise Alan Turing's Ph.D.

<img src="imgs/lambda.png">

*Image: the paper that introduced the lambda calculus. Not the easiest reading.*

### The rules

These are our rules:

* We *only* deal with functions. No integers, strings, or anything else. No loops, no (mutable) variables.
* We *only* deal with pure functions of one argument, which return one value.

That's enough to do *anything*, and perhaps a bit unlike other universal models of computation like Turing machines, it is practical to write simple programs in the bare lambda calculus.

<div class="alert alert-info">
    
In fact, in 2022, Hikaru Ikuta ("Woodrush") implemented an entire programming language, a version of Lisp, in the pure untyped lambda calculus: https://woodrush.github.io/blog/lambdalisp.html
</div>

We'll initially bend these rules a bit (e.g. to make it possible to print things out so we can see them!) but that's our basic principles.

<div class="alert alert-info">
    
**NOTE** This exercise is shamelessly adapted from [David Beazley's excellent talk on the lambda calculus in Python](https://www.youtube.com/watch?v=pkCLMl0e_0k). If you want to see this worked out in more depth, **watch that talk**!
    
</div>
    
Following David's notation, I will use lowercase letters for variables and all UPPERCASE letters for lambda terms that we define.    

### Notation station

In the standard lambda calculus notation, we write a function that takes one argument and returns a value like this:

    λx.x
    
`λx` can be thought of the parameter definition, `x`, as the return value when this expression is evaluated.

In Python, we'd write this:

    lambda x: x
    
This seems a very boring function (the identity function).

In [ ]:
IDENTITY = lambda x: x

In [ ]:
# test it works
IDENTITY("hey")

### Booleans as functions

We can write other functions in this notation:

    λxλy.y
    
This now has two "variables" `x` and `y`. We can write this in Python:

    lambda x: lambda y: y
    
What does this do? It is a function that returns a function, that returns the argument given to the second function; the first argument is discarded. (We can imagine this as the *curried* form of a two argument function, as we discussed in Part A).

In [ ]:
fy = lambda x: lambda y: y

# example
fy(1)(2)

In [ ]:
# This is just the same as writing

def fy(x):
    def fy_inner(y):
        return y
    return fy_inner

fy(1)(2)

Likewise, we can define:

    λxλy.x


In [ ]:
fx = lambda x: lambda y: x
fx(1)(2)

These two "terms" are sometimes called `true` and `false`. TRUE evaluates to the first argument, and FALSE to the second (obviously, we only have one real argument at a time, but we can see the sequential calls as being equivalent to two arguments). We can use them like below:

In [ ]:
TRUE =  lambda x: lambda y: x
FALSE =  lambda x: lambda y: y

# hey, it's a Boolean!
TRUE(True)(False)

This actually works like `if`/`else`. Given any two functions, we are choosing one or the other to evaluate.

In [ ]:
TRUE("It was true!")("It was not!")

### Things get more interesting: logical operators
We can, from this definition, define **logical operations**:

In [ ]:
AND = lambda x: lambda y: x(y)(x)

Spend a moment thinking about *why* this works.

    AND = λxλy.x(y)(x)
    
If we pass x=TRUE y=TRUE

    TRUE(TRUE)(TRUE)
    
Which is TRUE by our rule above. Likewise:

        x      y
        TRUE  TRUE       TRUE(TRUE)(TRUE)    = TRUE
        TRUE  FALSE      TRUE(FALSE)(TRUE)   = FALSE
        FALSE TRUE       FALSE(TRUE)(FALSE)  = FALSE
        FALSE FALSE      FALSE(FALSE)(FALSE) = FALSE


In [ ]:
# print a wee truth table

print("AND")
print("T T ->", AND(TRUE)(TRUE)("T")("F"))
print("F T ->", AND(FALSE)(TRUE)("T")("F"))
print("T F ->", AND(TRUE)(FALSE)("T")("F"))
print("F F ->", AND(FALSE)(FALSE)("T")("F"))

This is something special. We have defined a logical operation as a function operating on functions. 

## Task

* (a) Write a function `truth_table(lambda_op, title)` to print out the truth table for a lambda term like above, using a loop or comprehension, rather than manually. For example, `truth_table(AND, "AND")` should print the table above.

* (b) Define the operators OR, NOT, NAND, NOR and XOR in the same way and verify their truth tables.

In [ ]:
## Solution

def truth_table(lambda_op, title):    
    print(title)
    ops = [("T", TRUE), ("F", FALSE)]    
    print("\n".join([f"{a} {b} -> {lambda_op(lambda_a)(lambda_b)('T')('F')}" for a,lambda_a in ops for b,lambda_b in ops]))
    print()
    
truth_table(AND, "AND")    

OR = lambda x: lambda y: y(y)(x)
NOT = lambda x: x(FALSE)(TRUE)
NAND = lambda x: lambda y: NOT(AND(x)(y))
NOR = lambda x: lambda y: NOT(OR(x)(y))
XOR = lambda x: lambda y: AND(OR(x)(y))(NAND(x)(y))

truth_table(OR, "OR")
truth_table(NAND, "NAND")
truth_table(NOR, "NOR")
truth_table(XOR, "XOR")

# One...Two...Three

Let's try something a little more complicated. The letters, like `x` that we use don't matter; we can use any names we want. Here's another term:

    λkfx.kf(fx)
    
What does this look like in Python? When two variables appear next to each other in the lambda calculus, the convention is that we are *calling* that function, so we really have:

    λkfx.(k(f)(f(x)))
    
And the first part `λkfx.` just means we define three variables `k`, `f` and `x`, one after the other, the shorthand way of writing:

    λkλfλx.(k(f)(f(x)))
    
I'll call this term NEXT (sometimes called SUCC or SUCCESSOR)

    NEXT = λkfx.kf(fx) = λkλfλx.(k(f)(f(x)))
    

In [ ]:
NEXT = lambda k: lambda f: lambda x: k(f)(f(x))
ZERO = lambda f: lambda x: x # same as FALSE

This is an interesting pair of functions. In particular, imagine we called it like this:

In [ ]:
inc = lambda x: x + 1
ZERO(inc)(0)

In [ ]:
print(NEXT(ZERO)(inc)(0))
print(NEXT(NEXT(ZERO))(inc)(0))

In fact, we could define some abstract kind of numbers this way:    

In [ ]:
ONE = NEXT(ZERO)
TWO = NEXT(ONE)
THREE = NEXT(TWO)
FOUR = NEXT(THREE)
FIVE = NEXT(FOUR)
SIX = NEXT(FIVE)
SEVEN = NEXT(SIX)
EIGHT = NEXT(SEVEN)
NINE = NEXT(EIGHT)

In [ ]:
print(SIX(inc)(0))

This is quite an abstract form of number. It doesn't depend on the use of integers, for example. It encodes the idea of "six times" directly in functions instead. These are [**Church encoded numerals**](https://en.wikipedia.org/wiki/Church_encoding).

It captures the idea of six repetitions of an action. We don't have to use it as an integer value:

In [ ]:
print(SIX(lambda x: x+"Ho! ")(""))

In [ ]:
print(SIX(lambda x: x*2)(1)) # 2^6 = 64

We can define a couple of plain Python functions to convert to and from this notation:

In [ ]:
# a convenient function

inc = lambda x: x+1 # convenience

def print_church(church):
    # print a number out
    print(church(inc)(0))
    
def make_church(n):
    church = ZERO
    for i in range(n):
        church = NEXT(church)
    return church    
    

In [ ]:
print_church(make_church(8))
print_church(SEVEN)

Can we go further? Can we encode *arithmetic* using just functions? **Yes, we can.** Consider what adding is `x+y` -- `y` repeats of incrementing a number `x`; or do "NEXT x" y times. In the notation:

    λxy.(x NEXT y)

=

    λxλy.(x (λkλfλz.k f (f z)) y)

## Task
Implement this in Python to define ADD(x)(y)

In [ ]:
## Solution
ADD = lambda x: lambda y: x(NEXT)(y)

# or, expanding it out
ADD = lambda x: lambda y: x(lambda k: lambda f: lambda z: k(f)(f(z)))(y)

In [ ]:
print_church(ADD(FIVE)(SIX))
print_church(ADD(THREE)(ZERO))
print_church(ADD(ADD(ONE)(NINE))(NINE))

## Task

Write **multiplication** as a lambda term MUL(x)(y).

In [ ]:
## Solution
MUL = lambda x: lambda y: x(ADD(y))(ZERO)

In [ ]:
assert MUL(FOUR)(ZERO)(inc)(0)==0
assert MUL(THREE)(ONE)(inc)(0)==3
assert MUL(FOUR)(FOUR)(inc)(0)==16
assert MUL(THREE)(FIVE)(inc)(0)==15

### A predicate

We can write a test `IS_ZERO` to see if a number is zero or not:

In [ ]:
IS_ZERO = lambda n: n(lambda x:FALSE)(TRUE)

print(IS_ZERO(ZERO)(True)(False))
print(IS_ZERO(ONE)(True)(False))
print(IS_ZERO(FIVE)(True)(False))

## Task

Write `IS_NONZERO` as a lambda term

In [ ]:
## Solution
IS_NONZERO = lambda x: NOT(IS_ZERO(x))

## A list?

What about compound data structures? We can even represent lists using just functions. Consider representing a pair of values:

    (HEAD, [TAIL])
    
(sometimes called a "cons cell" for historical reasons). 

* HEAD -- a single value, like a number.
* [TAIL] -- the rest of the values, or nothing.

Any list could be represented by nesting these. For example, `["a", "b", "c"]` could be represented as:

    L = "a", ["b", ["c", ["d", END]]]
        
* HEAD(L) = "a"
* HEAD(TAIL(L)) = "b"
* HEAD(TAIL(TAIL(L))) = "c"

and so on.

We can write these list construction/decomposition functions directly as lambda terms:

    JOIN = λaλxλnλc.cax
    HEAD = λx.x(λaλb.b)(λaλx.a)
    TAIL = λx.x(λaλb.b)(λaλx.x)
   

In [ ]:
# join(a)(b) -> [a, b] b is a list; a is n element
JOIN = lambda a: lambda x: lambda nil: lambda cons: cons(a)(x)

# what tail will refer to at the end of a list
END = TRUE

# len(l) == 0?
EMPTY = lambda xs: xs(TRUE)(lambda a: lambda x: FALSE)

# head(l) -> l[0]
HEAD = lambda xs: xs(FALSE)(lambda a: lambda x: a)

# tail(l) -> l[1:]
TAIL = lambda xs: xs(FALSE)(lambda a: lambda x: x)

Let's test this with a "hand built" list

In [ ]:
MY_LIST = JOIN(SIX)(JOIN(TWO)(JOIN(FIVE)(END)))

In [ ]:
# hey, we can index!
print_church(HEAD(MY_LIST))
print_church(HEAD(TAIL((MY_LIST))))
print_church(HEAD(TAIL(TAIL(MY_LIST))))

## Task

* (a) Write an ordinary Python function `print_list()` to print out a list of integers in this format; and a function `make_list(l)` that takes an ordinary Python list of integers and returns a lambda list of Church numerals.
* (b) Write the **lambda term** REPZERO(n) that creates a list of *n* zeros and test that it works correctly.
* (c) Write the **lambda term** REP(k)(n) that creates a list of *n* copies of k and test that it works correctly.
* (d) Write the **lambda term** IOTA(n), that creates a list [n, n-1, ..., 1, 0] and test that it works correctly.

In [ ]:
## Solution

def print_list(lambda_list):
    print("[ ", end="")
    while not EMPTY(lambda_list)(True)(False):
        print(HEAD(lambda_list)(inc)(0), end=" ")
        lambda_list = TAIL(lambda_list)
    print("]")    
    
def make_list(l):
    l_list = END
    for elt in l[::-1]:        
        l_list = JOIN(make_church(elt))(l_list)
    return l_list
    

In [ ]:
print_list(make_list([2,3,4]))
print_list(JOIN(SIX)(JOIN(TWO)(JOIN(FIVE)(END))))
print_list(JOIN(ONE)(JOIN(FIVE)(END)))

In [ ]:
## Solution
REPZERO = lambda n: n(JOIN(ZERO))(END)
REP = lambda k: lambda n: n(JOIN(k))(END)

ZERO_LIST = (JOIN(ZERO)(END))
INC_HEAD = lambda x:JOIN(NEXT(HEAD(x)))(x)
IOTA = lambda n: n(INC_HEAD)(ZERO_LIST) 

In [ ]:
print_list(REPZERO(ZERO))
print_list(REPZERO(ONE))
print_list(REPZERO(TWO))
print_list(REPZERO(THREE))
print_list(REPZERO(FOUR))

In [ ]:
print_list(REP(FIVE)(THREE))
print_list(REP(TWO)(SIX))
print_list(REP(SIX)(TWO))

In [ ]:
print_list(IOTA(THREE))
print_list(IOTA(ONE))
print_list(IOTA(TWO))

### Recursion

So we can do **counted loops**. But what about **indefinite iteration**? What is the equivalent of `while`? We can't write `while` directly, but we *can* do **recursion**.

However, there is a slight fly in the ointment. Because of the way Python works, the obvious way to write a recursive function will cause an infinite loop. We can see the problem if we try and use TRUE and FALSE with `print`:

In [ ]:
# both are evaluated, and one returned
TRUE("True")("False")

In [ ]:
# both are evaluated, and one returned
# but this means both are still *executed*!
TRUE(print("True"))(print("False"))

To fix this (and it is just an artifact of the Python language, not of the lambda calculus itself), we can create "deferred" versions of TRUE and FALSE that do not evaluate immediately. 

In [ ]:
# note: we used a "dummy" lambda to wrap the expression in a function
# and a "dummy" function call to then evalute it
TRUE(lambda _: print("It was true"))(lambda _: print("It was false"))(TRUE)

This allows us to "wrap up" two branches following a predicate (like IS_ZERO) to prevent immediate infinite recursion.

In [ ]:
# D_ for "deferred"
D_TRUE = lambda x: lambda y: x(x)
D_FALSE = lambda x: lambda y: y(y)

# take a predicate and return the deferred version
DEFER = lambda pred: lambda x: pred(x)(D_TRUE)(D_FALSE)


In [ ]:
# NO: we need to use functions, that when called
# return the value
DEFER(EMPTY)(JOIN(ONE)(END))(True)(False)

In [ ]:
# YES: we use these "dummy" lambda terms instead (we never use the _ variable)
# it just stops the evaluation being done too "eagerly"
DEFER(EMPTY)(JOIN(ONE)(END))(lambda _:True)(lambda _:False)

Now we can write recursive functions, such as the *length of a list*.

In [ ]:
# the lambda _: is just a fix to the way Python interprets functions

LEN = lambda x: (
        DEFER(EMPTY)(x)  # len(x)==0?
            (lambda _: ZERO)    # base case: return 0
            (lambda _: NEXT(LEN(TAIL(x)))) # inductive case: return 1+len(l[1:])
        )

# test it works
print_church(LEN(END))
print_church(LEN(JOIN(ONE)(END)))
print_church(LEN(REPZERO(FOUR)))

print_church(LEN(make_list([3,1,4, 1,5,9,2])))

## Task
Write:    
* (a) **Indexing** A lambda term `NTH(l)(n)` that returns the Nth item of a list.
* (b) **Reduction example**. A lambda term `SUM(l)` that computes the *sum* of a list.
* (c) **Map example** A lambda term `MUL_LIST(l)(x)` that takes a constant `x` and multiplies each element of a list by it.
* (d) **Filter example** A lambda term `FILTER_ZERO(l)` that takes a list and returns every non-zero element of this list.
* (e) **Tricky example (!)** A lambda term `REVERSE` that reverses a list



In [ ]:
## Solution

NTH = lambda l: lambda n: HEAD(n(TAIL)(l))
SUM =  lambda x: DEFER(EMPTY)(x) (lambda _:ZERO)(lambda _: ADD(HEAD(x))(SUM(TAIL(x))))
MUL_LIST =  lambda l: lambda n: DEFER(EMPTY)(l) (lambda _:l) (lambda _: JOIN(MUL(n)(HEAD(l)))(MUL_LIST(TAIL(l))(n)))

FILTER_ZERO = (lambda l: DEFER(EMPTY)(l) (lambda _:l)
               (lambda _: DEFER(IS_ZERO)(HEAD(l))
                    (lambda _:FILTER_ZERO(TAIL(l)))
                    (lambda _:JOIN(HEAD(l))(FILTER_ZERO(TAIL(l))))))


REV = (lambda a: lambda b: DEFER(EMPTY)(a)
                    (lambda _: b)
                    (lambda _:REV(TAIL(a))(JOIN(HEAD(a))(b))))
                   

REVERSE = lambda l:REV(l)(END)
        

In [ ]:
# more examples
list_6958 = (JOIN(SIX)(JOIN(NINE)(JOIN(FIVE)(JOIN(EIGHT)(END)))))
list_695 = (JOIN(SIX)(JOIN(NINE)(JOIN(FIVE)(END))))
list_6958_2 = (JOIN(SIX)(JOIN(NINE)(JOIN(FIVE)(JOIN(EIGHT)(END)))))
list_958 = (JOIN(NINE)(JOIN(FIVE)(JOIN(EIGHT)(END))))
list_6050 = JOIN(SIX)(JOIN(ZERO)(JOIN(FIVE)(JOIN(ZERO)(END))))


print_list(list_6958)
print_list(REVERSE(list_6958))

In [ ]:
assert NTH(list_6958)(ZERO) == SIX
assert NTH(list_6958)(ONE) == NINE
assert NTH(list_6958)(TWO) == FIVE
assert NTH(list_6958)(THREE) == EIGHT

assert SUM(IOTA(FOUR))(inc)(0) == 10
assert SUM(list_6958)(inc)(0) == 6+9+5+8
assert NTH(REVERSE(list_6958))(ZERO) == EIGHT
assert NTH(REVERSE(list_6958))(TWO) == NINE

mul_2 = MUL_LIST(list_6958)(TWO) 
assert NTH(mul_2)(ZERO)(inc)(0) == 12
assert NTH(mul_2)(ONE)(inc)(0) == 18


assert LEN(FILTER_ZERO(list_6050))(inc)(0) == 2

## Task

Write the higher-order functions that work with *any* passed in function:
    
* REDUCE(fn)(l)(init) reduces fn over l, starting with an initial value init (e.g. REDUCE(ADD)(l)(0) is SUM(l))
* MAP(fn)(l) applies fn to each element of l, and returns the tranformed list
* FILTER(pred)(l) applies pred to each element of l, and returns the collected values where pred was True
* APPLY(fn)(args) applies fn with the sequence of arguments given (like `fn(*args)`)

In [ ]:
## Solution

MAP = lambda f: lambda l: DEFER(EMPTY)(l) (lambda _:END) (lambda _: JOIN(f(HEAD(l)))(MAP(f)(TAIL(l))))
REDUCE =  lambda f: lambda l: lambda init: DEFER(EMPTY)(l) (lambda _:init)(lambda _: f(HEAD(l))(REDUCE(f)(TAIL(l))(init)))

APPLY = (lambda f: lambda args: 
        DEFER(EMPTY)(args) (lambda _: FALSE) 
        (lambda _: 
             DEFER(EMPTY)(TAIL(args)) 
                 (lambda _: f(HEAD(args)))
                 (lambda _: APPLY(f(HEAD(args)))(TAIL(args)))))


            
FILTER = (lambda pred: lambda l:
            DEFER(EMPTY)(l) (lambda _:l)
               (lambda _: pred(HEAD(l))
                    (FILTER(pred)(TAIL(l)))
                    (JOIN(HEAD(l))(FILTER(pred)(TAIL(l))))))


In [ ]:
print_church(APPLY(ADD)(JOIN(ONE)(JOIN(THREE)(END))))

In [ ]:
list_695 = (JOIN(SIX)(JOIN(NINE)(JOIN(FIVE)(END))))

DOUBLE = lambda q: ADD(q)(q)

print_list(MAP(DOUBLE)(list_695))
print_church(REDUCE(ADD)(list_695)(ZERO))

In [ ]:
# redefine FILTER_ZERO using the general form
FILTER_ZERO_2 = lambda x: FILTER(IS_ZERO)(x)

# we can, for example, define ALL and ANY
ALL = lambda x: REDUCE(AND)(x)(TRUE)
ANY = lambda x: REDUCE(OR)(x)(FALSE)

print_list(FILTER_ZERO_2(list_6050))
           

## Task

* (a) Write a lambda term `EQ(a)(b)` that will test if two Church numerals are equal. **This is harder than it sounds!**. 
* (b) Generalise this to write LEQ (less than or equal to), GEQ (greater than or equal to), GT (greater than) and LT (less than).
* (c) Write a lambda term `EQ_LIST(a)(b)` that is tests if two lists contain the same numbers in the same order (easier than it sounds).

In [ ]:
## Solution

# obviously lots of ways of doing this. we'll use a unary list, instead of the standard
# PRED/SUBTRACT method

INC_UNARY = lambda l: JOIN(ONE)(l)
DEC_UNARY = lambda l: EMPTY(l)(l)(TAIL(l))
INC_DEC = lambda a: lambda b: a(DEC_UNARY)(b(INC_UNARY)(END))
LEQ = lambda a: lambda b: EMPTY(INC_DEC(b)(a))
EQ = lambda a: lambda b: AND(LEQ(a)(b))(LEQ(b)(a))
LT = lambda a: lambda b: AND(LEQ(a)(b))(NOT(LEQ(b)(a)))
GT = lambda a: lambda b: NOT(LEQ(a)(b))
GEQ = lambda a: lambda b: NOT(LT(a)(b))


EQ_LIST = (lambda a: lambda b: DEFER(EMPTY)(a) 
    (lambda _: EMPTY(b)(TRUE)(FALSE))
    (lambda _: AND(EQ(HEAD(a))(HEAD(b)))(EQ_LIST(TAIL(a))(TAIL(b)))))


In [ ]:
assert EQ(ONE)(ONE)(True)(False)
assert not EQ(ZERO)(ONE)(True)(False)
assert EQ(THREE)(THREE)(True)(False)
assert not EQ(FIVE)(SEVEN)(True)(False)
assert not EQ(SEVEN)(FIVE)(True)(False)
assert EQ(NEXT(FOUR))(NEXT(FOUR))(True)(False)
assert EQ(ZERO)(ZERO)(True)(False)

In [ ]:
assert LEQ(THREE)(FOUR)(True)(False)
assert LEQ(THREE)(THREE)(True)(False)
assert not LT(THREE)(THREE)(True)(False)
assert not GT(THREE)(FOUR)(True)(False)
assert GT(NINE)(FOUR)(True)(False)
assert GEQ(NINE)(FOUR)(True)(False)
assert GEQ(FOUR)(FOUR)(True)(False)
assert not GT(FOUR)(FOUR)(True)(False)

In [ ]:
assert not EQ_LIST(list_6958)(list_6050)(True)(False)
assert not EQ_LIST(list_6958)(list_695)(True)(False)
assert not EQ_LIST(list_6958)(list_958)(True)(False)
assert not EQ_LIST(list_6958)(END)(True)(False)
assert EQ_LIST(list_6958)(list_6958)(True)(False)
assert EQ_LIST(list_6958)(list_6958_2)(True)(False)

### Z combinator

I'll introduce one further lambda term. This one is hard to explain concisely, but it allows us to write *recursive functions* as a pure lambda term. So far, we've used things like: 

    LEN = lambda x: (
            D_EMPTY(x)  # len(x)==0?
                (lambda _: ZERO)    # base case: return 0
                (lambda _: NEXT(LEN(TAIL(x)))) # inductive case: return 1+len(l[1:])
            )

**But this is cheating!** We've used the variable `LEN` in the right hand side. This presumes we have some additional storage (local variables) to make this reference. It's not inside our rules for lambda calculus. **We can't use a recursive reference like this if we are doing things properly.** The other references to earlier definitions  (like `NEXT` or `ZERO`) are fine; they are just a convenience for substituting in earlier definitions instead of writing it out each time. We could write them out long-hand if we wanted, but we can't write out a self-reference long-hand.

What can we do? We can use a lambda term that allows *self reference*. One of these is known as the **Z combinator** or the **fixed point combinator**. This lets us repeat operations via recursion without any external variables!

    λf(λx.f(λz.x(x)(z))(λx.f(λz.x(x)(z))
    
Using this, we can pass the recursive function in *and* get a reference to it.    
    

In [ ]:
Z = lambda f:(lambda x: f(lambda z:x(x)(z)))(lambda x:f(lambda z:x(x)(z)))

In [ ]:
# note: f becomes our reference to the recursive call
# when we use Z like this

LEN_Z = Z(lambda f: lambda x: (
            DEFER(EMPTY)(x)  # len(x)==0?
                (lambda _: ZERO)    # base case: return 0
                (lambda _: NEXT(f(TAIL(x)))) # inductive case: return 1+len(l[1:])
            ))


print_church(LEN_Z(JOIN(SIX)(JOIN(NINE)(JOIN(FIVE)(END)))))

## Task: Use Z

* Rewrite SUM and REVERSE using Z instead of refering to SUM/REVERSE in its definition.

In [ ]:
## Solution
SUM_Z =  Z(lambda f:lambda x: DEFER(EMPTY)(x) (lambda _:ZERO)(lambda _: ADD(HEAD(x))(f(TAIL(x)))))

REV_Z = Z(lambda f: lambda a: lambda b: DEFER(EMPTY)(a)
                    (lambda _: b)
                    (lambda _:f(TAIL(a))(JOIN(HEAD(a))(b))))
                   

REVERSE_Z = lambda l:REV_Z(l)(END)
        

print_church(SUM_Z(list_6958))
print_list(REVERSE_Z(list_6958))


## Task A.3 Redux

Now, redo Task A.3 (the dot product) using only the lambda calculus. No Python lists, integers or anything other than the pure one argument functions we have been defining so far.

Define DOT_PRODUCT(a)(b) that takes two lambda lists `a` and `b` and returns the dot product.

You may assume `a` and `b` will always be the same length


In [ ]:
## Solution
DOT_PRODUCT = Z(lambda f: lambda a: lambda b:
    DEFER(EMPTY)(b) (lambda _: ZERO)
                    (lambda _: ADD(MUL(HEAD(a))(HEAD(b)))
                                  (f(TAIL(a))(TAIL(b)))))

In [ ]:
## Tests
A = make_list([1,2,5])
B = make_list([2, 0, 5])            
C = make_list([5, 3, 1, 1])
D = make_list([1, 2, 3 , 4])

assert DOT_PRODUCT(A)(B)(inc)(0) == 1*2+2*0+5*5
assert DOT_PRODUCT(C)(D)(inc)(0) == 5*1+3*2+1*3+1*4

Now you should be convinced that all the luxuries of Python -- lists, multiple arguments, comprehensions, even integers themselves are just that: *luxuries*. Functions are all you need.

Note that we used the *untyped* lambda calculus. Most functional programming is based on the **typed lambda calculus**, where we give types to the arguments that the functions can take. This has many benefits, not least making it possible to prove attributes of programs mathematically before they are executed.

If you start with the typed lambda calculus, add a bit of syntatic sugar and some optimisations, then you've got... Haskell.

If you are interested:

* [An interesting blog post and interpreter](https://crypto.stanford.edu/~blynn/lambda/)
* [A Brief and Informal Introduction to the Lambda Calculus](https://www.cs.yale.edu/homes/hudak/CS201S08/lambda.pdf)
* [The impact of lambda calculus in logic and computer science](https://www-users.mat.umk.pl//~adwid/materialy/doc/church.pdf)


## Challenge exercise

Write **MERGE_SORT** as a lambda term :), so you can sort lists of Church numerals!



In [ ]:
## Solution

ONE_HEAD = lambda f: lambda a: lambda b: JOIN(HEAD(a))(f(TAIL(a))(b))

# len(l)<2
LEN_ONE = lambda l: EMPTY(l)(TRUE)(EMPTY(TAIL(l))(TRUE)(FALSE))

MERGE = (lambda a: lambda b: 
            DEFER(EMPTY)(a)  # len(a)==0?
                  (lambda _:b) # return b
                  (lambda _:DEFER(EMPTY)(b) # len(b) == 0?
                           (lambda _:a) # return a
                           (lambda _: LT(HEAD(a))(HEAD(b)) # a[0]<b[0]
                                            (ONE_HEAD(MERGE)(a)(b))
                                            (ONE_HEAD(MERGE)(b)(a)))))
# split into even/odd pairs
SPLIT = (lambda l: lambda flag: 
         DEFER(EMPTY)(l)(lambda _: l) # len(l)==0: return l
            (lambda _: flag(  # if flag:
                        JOIN(HEAD(l))(SPLIT(TAIL(l))(NOT(flag)))) # return l[0] + split(l[1:], not flag)
                        (SPLIT(TAIL(l))(NOT(flag))))) # return split(l[1:], not flag)

MERGE_SORT = (lambda l: DEFER(LEN_ONE)(l) # len(l)<2?
                (lambda _: l) # return l
                (lambda _: # merge(merge_sort(left(l), right(l)))
                    MERGE(MERGE_SORT(SPLIT(l)(TRUE)))
                         (MERGE_SORT(SPLIT(l)(FALSE)))))


In [ ]:
print_list(MERGE_SORT(make_list([0,1,4,1,3,3,0,1,6,5,0])))